In [57]:
import os
import sys
import random
from scipy import signal
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# get filename_list
matplotlib.use('Agg')

In [58]:
import scipy.io
mat = scipy.io.loadmat('../data/preprocessed/hiroo-cnn-own.mat')
## load
eeg = mat["eegData"]
emotions = mat["labels"]
emotions = emotions.astype(int)
nchannel = eeg.shape[1]
ntrial = eeg.shape[2]

In [23]:
eeg.shape

(29440, 29, 3)

In [24]:
emotions.shape

(29440, 3)

In [59]:
def dataAugmentation(matrix, labels, DATAPOINT, STRIDE_SIZE, fs): ## able to improve
    n_size = int((matrix.shape[0] - DATAPOINT) // STRIDE_SIZE) + 1
    augmentedMat = np.empty((n_size*matrix.shape[2], 1, DATAPOINT, matrix.shape[1]))
    augmentedLabel = np.empty((n_size*matrix.shape[2]))
    print(matrix.shape[-1], n_size, matrix.shape)
    for t in range(matrix.shape[-1]):
        trial = matrix[:, :, t]
        for i in range(n_size):
            augmentedMat[i+t*n_size, :, :, :] = trial[i*STRIDE_SIZE:i*STRIDE_SIZE+DATAPOINT, :]
            augmentedLabel[i+t*n_size] =  np.mean(labels[i*STRIDE_SIZE:i*STRIDE_SIZE+DATAPOINT, t])
    return augmentedMat, augmentedLabel

In [60]:
ans, lab = dataAugmentation(eeg, emotions, 640, 128, 128)

3 226 (29440, 29, 3)


In [27]:
emotions

array([[2, 4, 1],
       [2, 4, 1],
       [2, 4, 1],
       ...,
       [1, 4, 1],
       [1, 4, 1],
       [1, 4, 1]])

In [61]:
import torch
import torch.nn as nn
from scipy import stats
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

class EmotionDataManager(Dataset):
    def __init__(self, data, label):
        self.df = data
        self.label = label
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        label = torch.tensor(self.label[index]).float()
        data = torch.tensor(self.df[index]).float()
        
        return data, label

In [62]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        ## (1, 640, 28)
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=(16, 8), stride=(4,2), padding=(6,3)),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True),
                                   nn.MaxPool2d(1)
                                  )
        ## (16, 160, 14)
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=(16,4) , stride=(4,2), padding=(6, 1)),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True),
                                   nn.MaxPool2d(1),
                                  )
        
        ## (32, 40, 7)
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=(8,3) , stride=(4,3), padding=(2, 1)),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True),
                                   nn.MaxPool2d(1),
                                  )
        ## (64, 10, 3)
#         self.conv4 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=(8,1) , stride=(2,1), padding=(3, 0)),
#                                    nn.BatchNorm2d(64),
#                                    nn.ReLU(inplace=True),
#                                    nn.MaxPool2d(1),
#                                   )
        ## (64, 40,3)
        self.dropout1 = torch.nn.Dropout(p=0.50)
#         self.dropout2 = torch.nn.Dropout(p=0.50)
        self.fc1 = nn.Sequential(nn.Linear(64 * 10 *  3, 128),nn.ReLU(inplace=True),)
#         self.fc2 = nn.Sequential(nn.Linear(1024, 64),nn.ReLU(inplace=True),)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
#         x = self.conv4(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x= self.dropout1(x)
#         x = self.fc2(x)
#         x= self.dropout2(x)
#         print(x[:, :5])
        x = self.fc3(x)
        return x

In [63]:
def acc(outputs, labels, bw):
    outputs = outputs.reshape(-1)
    labels = labels.reshape(-1)
    c1 = outputs > labels-bw
    c2 = outputs < labels+bw
    result = np.zeros(c1.shape, dtype=bool)
    for i in range(len(c1)):
        if c1[i]  and c2[i]:
            result[i] = True        
    
    return result

In [64]:
## Input
fs = 128
tw = 5
DATAPOINT = fs*tw
STRIDE_SIZE = fs//4
X, Y = dataAugmentation(eeg, emotions, DATAPOINT, STRIDE_SIZE, fs)

3 901 (29440, 29, 3)


In [65]:
mat = scipy.io.loadmat('../data/preprocessed/hiroo-cnn.mat')
## load
eeg = mat["eegData"]
emotions = mat["labels"]
emotions = emotions.astype(int)
nchannel = eeg.shape[1]
ntrial = eeg.shape[2]
X2, Y2 = dataAugmentation(eeg, emotions, DATAPOINT, STRIDE_SIZE, fs)

10 181 (6400, 29, 10)


In [66]:
X2.shape, X.shape
X = np.concatenate((X,X2), 0)
Y = np.concatenate((Y,Y2), 0)

In [67]:

X.shape, Y.shape

((4513, 1, 640, 29), (4513,))

In [69]:
batch_size = 32
n_splits = 5
studyRate = 0.002
kf = KFold(n_splits=n_splits, shuffle=True)
description = "hiroo-" +str(DATAPOINT)+"-"+str(STRIDE_SIZE)+"-lr-"+str(studyRate)

# datasize=range(ntrial)
datasize = range(len(X))
trail = 10

train_loss_value=np.empty((n_splits, trail))
train_acc_value=np.empty((n_splits, trail))
test_loss_value=np.empty((n_splits, trail))
test_acc_value=np.empty((n_splits, trail))
test_acc_each=np.zeros((n_splits, trail, 5))

for fold, (train_index, test_index) in enumerate(kf.split(datasize)):
#     train_x, train_y = dataAugmentation(eeg[:, :, train_index], emotions[:, train_index], DATAPOINT, STRIDE_SIZE, fs)
#     test_x, test_y = dataAugmentation(eeg[:, :, test_index], emotions[:, test_index,], DATAPOINT, STRIDE_SIZE, fs)
    train_x, train_y = X[train_index], Y[train_index]
    test_x, test_y = X[test_index], Y[test_index]
    traindataset = EmotionDataManager(train_x, train_y)
    testdataset = EmotionDataManager(test_x, test_y)
    trainloader = DataLoader(traindataset, batch_size,shuffle=True, num_workers=0)
    testloader = DataLoader(testdataset, batch_size,shuffle=True, num_workers=0)
    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = CNN()
    model = net.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=studyRate)
    for epoch in range(trail):
        print('epoch', epoch+1)

        sum_loss = 0.0
        sum_correct = 0
        sum_total = 0

        net.train()
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
#             print("===============")
#             print(i)
#             print("===============")
            loss = criterion(outputs, labels)
#             l2_lambda = 0.0001
#             l2_norm = sum(p.pow(2.0).sum()
#                           for p in model.parameters())

#             loss = loss + l2_lambda * l2_norm
            loss.backward()
            optimizer.step()
            ## softmaxをかませる
            sum_loss += loss.item()                     
            sum_total += labels.size(0)                 
            sum_correct += acc(outputs, labels, 0.5).sum().item()

        print("train mean loss={}, accuracy={}".format(sum_loss*batch_size/len(trainloader.dataset), float(sum_correct/sum_total)))
        train_loss_value[fold, epoch] = sum_loss*batch_size/len(trainloader.dataset)
        train_acc_value[fold, epoch] = float(sum_correct/sum_total)

        sum_loss = 0.0
        sum_correct = 0
        sum_total = 0
        
#         label_each = np.zeros(5)
#         acc_each = np.zeros(5)

        net.eval()
        for i, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            sum_loss += loss.item()                     
            sum_total += labels.size(0)                 
            sum_correct += acc(outputs, labels, 0.5).sum().item()
            
#             for l, lab in enumerate(labels):
#                 label_each[lab] += 1
#                 if (predicted[l] == lab):
#                     acc_each[lab] += 1

        print("test  mean loss={}, accuracy={}".format(sum_loss*batch_size/len(testloader.dataset), float(sum_correct/sum_total)))
        test_loss_value[fold, epoch] = sum_loss*batch_size/len(testloader.dataset)
        test_acc_value[fold, epoch] = float(sum_correct/sum_total)
#         for lab in range(5):
#             if  label_each[lab] > 0:
#                 test_acc_each[fold, epoch, lab] = acc_each[lab] / label_each[lab]


epoch 1
train mean loss=2.151967259457237, accuracy=0.1994459833795014
test  mean loss=1.7880101378177884, accuracy=0.1362126245847176
epoch 2
train mean loss=2.06078685749931, accuracy=0.21578947368421053
test  mean loss=1.76112846045003, accuracy=0.2779623477297896
epoch 3
train mean loss=1.9844482125998204, accuracy=0.1994459833795014
test  mean loss=1.7723416109813803, accuracy=0.2956810631229236
epoch 4
train mean loss=1.9929241000780438, accuracy=0.1961218836565097
test  mean loss=1.7227103063302447, accuracy=0.13953488372093023
epoch 5
train mean loss=1.9858956397735512, accuracy=0.1997229916897507
test  mean loss=1.7918803953253153, accuracy=0.12181616832779624
epoch 6
train mean loss=1.9639049794205008, accuracy=0.2027700831024931
test  mean loss=1.708749811250638, accuracy=0.1273532668881506
epoch 7
train mean loss=1.938202067029113, accuracy=0.19889196675900278
test  mean loss=1.7134661373505957, accuracy=0.17940199335548174
epoch 8
train mean loss=1.930965872682693, accurac

/Users/hiroo/opt/miniconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


train mean loss=2.223566067446569, accuracy=0.20963721960675713


/Users/hiroo/opt/miniconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test  mean loss=1.9522346911039161, accuracy=0.25055432372505543
epoch 2
train mean loss=2.062042019764032, accuracy=0.2079756300193852
test  mean loss=1.8322894018135156, accuracy=0.24057649667405764
epoch 3
train mean loss=2.024869899279653, accuracy=0.20299086125726945
test  mean loss=1.8171783349995074, accuracy=0.12971175166297116
epoch 4
train mean loss=1.9932193349385585, accuracy=0.20852949321517586
test  mean loss=1.8515623688962137, accuracy=0.1286031042128603
epoch 5
train mean loss=2.0244540497392407, accuracy=0.2104680144004431
test  mean loss=1.842446756468644, accuracy=0.13082039911308205
epoch 6
train mean loss=1.9707136447328637, accuracy=0.20382165605095542
test  mean loss=1.8142380259782407, accuracy=0.12971175166297116
epoch 7
train mean loss=1.9754956482322352, accuracy=0.19855995569094434
test  mean loss=1.828904280905713, accuracy=0.25055432372505543
epoch 8
train mean loss=1.9635876212150427, accuracy=0.2093602880088618
test  mean loss=1.8291832866795577, accura

In [167]:
bw=0.5
outputs = outputs.reshape(-1)
labels = labels.reshape(-1)
c1 = outputs > labels-bw
c2 = outputs < labels+bw
result = np.zeros(c1.shape, dtype=bool)
for i in range(len(c1)):
    if c1[i]  and c2[i]:
        result[i] = True        

In [180]:
plt.hist(test_y)
plt.savefig('../results/hiroo/cnn/hist.jpg')

In [56]:
outputs, labels

(tensor([[2.0444],
         [2.1898],
         [1.8879],
         [2.1043],
         [2.3795],
         [2.4689],
         [2.0080],
         [2.5236],
         [2.7248],
         [1.8992],
         [2.3171],
         [2.3911],
         [2.4274],
         [2.6134],
         [3.0121],
         [2.5251],
         [2.2651],
         [2.2157],
         [2.8942],
         [2.3761],
         [2.9083],
         [1.9333],
         [1.7129],
         [2.5337],
         [2.0966],
         [2.9140],
         [2.2911],
         [2.0485],
         [1.8254],
         [2.4503],
         [2.4862],
         [2.0489]], grad_fn=<AddmmBackward0>),
 tensor([5.0000, 5.0000, 3.0000, 1.0000, 4.0000, 3.0000, 1.0000, 1.0000, 4.0000,
         3.9000, 5.0000, 2.0000, 4.0000, 3.0000, 1.0000, 1.0000, 4.0000, 2.0000,
         5.0000, 4.0000, 4.0000, 1.0000, 1.0000, 2.0000, 1.0000, 1.0000, 1.0000,
         2.0000, 2.0000, 2.0000, 4.0000, 1.0000]))

In [187]:
acc(outputs, labels, 0.5)

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False])

In [30]:
np.all(a, 1)

array([ True, False])

In [189]:
for i in range(n_splits):
    matplotlib.use('Agg')

    fig = plt.figure(figsize=(30, 10))
    ax_right = fig.add_subplot(121)
    ax_left = fig.add_subplot(122)

    x = np.linspace(0, test_loss_value.shape[1], test_loss_value.shape[1], endpoint=False)
    ax_right.plot(x, train_loss_value[i, :], label="train")
    ax_right.plot(x, test_loss_value[i, :], label="test")
    ax_right.set_title("Loss at each epoch",fontsize=32)
    ax_right.set_xlabel("Epoch",fontsize=32)
    ax_right.set_ylabel("Loss",fontsize=32)
    ax_right.tick_params(axis='x', labelsize=24)
    ax_right.tick_params(axis='y', labelsize=24)
    ax_right.legend()


    ax_left.plot(x, train_acc_value[i, :], label="train")
    ax_left.plot(x, test_acc_value[i, :], label="test")
    ax_left.set_title("Accuracy at each epoch",fontsize=32)
    ax_left.set_xlabel("Epoch",fontsize=32)
    ax_left.set_ylabel("Accuracy rate",fontsize=32)
    ax_left.tick_params(axis='x', labelsize=24)
    ax_left.tick_params(axis='y', labelsize=24)
    ax_left.legend()

    plt.show()
    plt.savefig('../results/hiroo/cnn/1127-%s-fold-%d.jpg' % (description, i))

/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_14582/3456356805.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_14582/3456356805.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_14582/3456356805.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_14582/3456356805.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_14582/3456356805.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [46]:
acc(outputs, labels, 0.5)

array([ True,  True,  True,  True,  True,  True])

In [47]:
outputs

tensor([[2.9783],
        [2.9873],
        [3.1707],
        [3.2907],
        [2.9589],
        [3.4052]], grad_fn=<AddmmBackward0>)

In [48]:
labels

tensor([3.1000, 3.1000, 3.2000, 3.1100, 3.1000, 3.1000])